In [85]:
import os
import cv2
import albumentations as A
import numpy as np
from tqdm import tqdm

In [66]:
with open('./classes.txt') as f:
    classes = f.read().splitlines()

# classes

In [67]:
pump_labels = os.listdir("pump/labels copy")
pump_images = os.listdir("pump/images copy")


# sort the labels and images
pump_labels.sort()
pump_images.sort()

In [68]:
# Function to read YOLO-formatted label file
def read_yolo_label(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    bboxes = []
    class_labels = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        bboxes.append([x_center, y_center, width, height])
        class_labels.append(int(class_id))
    return bboxes, class_labels

# Function to save YOLO-formatted label file
def save_yolo_label(label_path, bboxes, class_labels):
    with open(label_path, 'w') as file:
        for bbox, class_id in zip(bboxes, class_labels):
            x_center, y_center, width, height = bbox
            file.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


In [88]:
# Paths
input_images_dir = './pump/images copy/'
input_labels_dir = './pump/labels copy/'
output_images_dir = './augmented_images/'
output_labels_dir = './augmented_labels/'

# Ensure output directories exist
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

In [89]:
# Define the augmentation pipeline
transform = A.Compose(
    [
        # A.RandomCrop(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.6),
        A.BBoxSafeRandomCrop(erosion_rate=0.0, p=0.6),
        # A.Morphological(),
        # A.PixelDropout(p=0.1),
        # A.RGBShift(p=0.1),
        A.VerticalFlip(p=0.5),
        A.Transpose(p=0.6),
        A.OpticalDistortion(p=0.6),
        A.FrequencyMasking(p=0.6),
        A.GaussNoise(p=0.6),
        # A.GridDistortion(p=0.6),
        # A.RandomGridShuffle(p=0.6),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])
)


In [91]:

# Process each image and its corresponding label
for image_name in tqdm(os.listdir(input_images_dir)):
    if image_name.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(input_images_dir, image_name)
        label_path = os.path.join(input_labels_dir, os.path.splitext(image_name)[0] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to open image file {image_path}")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, _ = image.shape

        # Read labels
        if not os.path.exists(label_path):
            print(f"Warning: Label file {label_path} does not exist.")
            continue
        bboxes, class_labels = read_yolo_label(label_path)
        
        # Apply augmentations 5 times
        for i in range(20):
            transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            transformed_image = transformed['image']
            transformed_bboxes = transformed['bboxes']
            transformed_class_labels = transformed['class_labels']

            # Generate new file names
            base_name = os.path.splitext(image_name)[0]
            new_image_name = f"{base_name}_aug_{i}.jpg"
            new_label_name = f"{base_name}_aug_{i}.txt"

            # Save augmented image
            transformed_image_bgr = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(output_images_dir, new_image_name), transformed_image_bgr)

            # Save augmented label
            save_yolo_label(os.path.join(output_labels_dir, new_label_name), transformed_bboxes, transformed_class_labels)

print("Dataset augmentation completed.")



100%|██████████| 64/64 [00:19<00:00,  3.26it/s]

Dataset augmentation completed.


In [92]:
# Move all from ./augmented_images to ./images

# Paths
output_images_dir = './images/'
output_labels_dir = './labels/'

input_images_dir = './augmented_images/'
input_labels_dir = './augmented_labels/'



In [93]:
input_images = os.listdir(input_images_dir)
input_labels = os.listdir(input_labels_dir)

# sort the labels and images
input_images.sort()
input_labels.sort()

for i in range(len(input_images)):
    os.rename(input_images_dir + input_images[i], output_images_dir + input_images[i])
    os.rename(input_labels_dir + input_labels[i], output_labels_dir + input_labels[i])
